In [ ]:
import sqlalchemy
from sqlalchemy import select
from datetime import datetime
from pathlib import Path

from fastai.vision.all import *
from watcher.training import *

from watcher import *

## Load and explore data

In [ ]:
tc = TunneledConnection()
session = sqlalchemy.orm.Session(tc)

In [ ]:
unique_labels = UniqueClassificationLabels(session)
unique_labels

In [ ]:
stmt = (select(EventClassification,Computation)
        .join(EventObservation, EventObservation.id == EventClassification.observation_id)
        .join(Computation, EventObservation.event_name == Computation.event_name)
        .where(EventClassification.confidence == None)
        .where(Computation.result_file != None)
       )

results = session.execute(stmt)

labelings = {}
for r in results:
    name = r[1].event_name
    file = Path(r[1].result_file_location) / r[1].result_file
    item = labelings.get(name,{
        'file': file,
        'lbls': []
    })
    item['lbls'].append('noise' if r[0].label.startswith('noise') else r[0].label)
    labelings[name] = item

data = [[k, str(v['file']), ' & '.join(sorted(v['lbls']))] for k, v in labelings.items()]
    

df = pd.DataFrame(data=data, columns=['event_name','file','label'])
df

In [ ]:
session.close()
tc.disconnect()

## Train a model

In [ ]:
root = !pwd
base_path = Path(root[0]) / application_config('system','LOCAL_DATA_DIR')
base_path

In [ ]:
dls = ImageDataLoaders.from_df(df, base_path, label_delim=' & ',
                               fn_col='file',
                               label_col='label',
                               item_tfms=Resize(460),
                               batch_tfms=aug_transforms(size=224))

In [ ]:
dls.show_batch()

from the [fast.ai tutorial](https://docs.fast.ai/tutorial.vision.html#multi-label-classification---using-the-high-level-api):

In [ ]:
f1_macro = F1ScoreMulti(thresh=0.5, average='macro')
f1_macro.name = 'F1(macro)'
f1_samples = F1ScoreMulti(thresh=0.5, average='samples')
f1_samples.name = 'F1(samples)'
learn = vision_learner(dls, resnet50, metrics=[partial(accuracy_multi, thresh=0.5), f1_macro, f1_samples])

In [ ]:
learning_rate = learn.lr_find()
learning_rate

In [ ]:
learn.fine_tune(15, learning_rate.valley)

## Qualitative inspection

i.e. take a look!

In [ ]:
learn.show_results()

### run a few samples

need some better utilities here.

## Save it!

Filename goes in application.cfg to use

In [ ]:
timestamp = datetime.now().isoformat() 
modeldir = Path(root[0]) / 'etc' 
filepath = str( modeldir / f"model-{timestamp}.fastai.pkl")
learn.export(filepath)
filepath